In [1]:
from __future__ import print_function
from imutils.object_detection import non_max_suppression
from imutils import paths
import numpy as np
import imutils
import cv2

In [2]:
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

In [3]:
video_path = r'C:\Users\user\Desktop\ML\CrowdDetection\RP1\input\input\video1.mp4'

In [4]:
cap = cv2.VideoCapture(video_path)
if (cap.isOpened() == False):
    print('Error while trying to open video. Please check again...')

In [5]:
video_outpath = r'C:\Users\user\Desktop\ML\CrowdDetection\RP1\input\output\output.mp4'

In [6]:
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
if frame_width < 400: 
        frame_width = 400
        ratio = frame_width / float(frame_width) 
        frame_height = int(frame_width * ratio)
out = cv2.VideoWriter(video_outpath, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width,frame_height))
frame_count = 0
total_fps = 0

In [7]:
import time
import os

In [8]:
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        start_time = time.time()
        
        frame = cv2.resize(frame, (frame_width, frame_height))
        img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects, weights = hog.detectMultiScale(img_gray, winStride=(4, 4), padding=(4, 4), scale=1.02)
        for i, (x, y, w, h) in enumerate(rects):
            if weights[i] < 0.13:
                continue
            elif weights[i] < 0.3 and weights[i] > 0.13:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            if weights[i] < 0.7 and weights[i] > 0.3:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (50, 122, 255), 2)
            if weights[i] > 0.7:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, 'High confidence', (10, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, 'Moderate confidence', (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (50, 122, 255), 2)
        cv2.putText(frame, 'Low confidence', (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        frame_write_name = 'frame'+str(frame_count)+'.jpg'
        file_path = r'C:\Users\user\Desktop\ML\CrowdDetection\RP1\input\output\frames'
        cv2.imwrite(os.path.join(file_path,frame_write_name), frame)
        
    
        end_time = time.time()
        fps = 1 / (end_time - start_time)
        total_fps += fps
        frame_count += 1
        cv2.imshow("Preview", frame)
        out.write(frame)
        wait_time = max(1, int(fps/4))
        if cv2.waitKey(wait_time) & 0xFF == ord('q'):
            break
    else:
        break


avg_fps = total_fps / frame_count
print(f"Average FPS: {avg_fps:.3f}" )
cap.release()
cv2.destroyAllWindows()

Average FPS: 6.517
